# DATA COLLECTION AND PREPARATION

### Install Spotipy to generate data

In [4]:
# !pip3 install spotipy

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 61 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 788 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 625 kB/s eta 0:00:01
     |████████████████████████████████| 137 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 2.5 MB/s eta 0:00:01


In [92]:
try:
    sc.stop()
except:
    pass

In [93]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Capstone_Project').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)

from pyspark.sql import SparkSession 
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
spark = SparkSession.builder.appName('Capstone_SQL').getOrCreate()

## GET DATA

Datasets will be in 2 parts: the track IDs and audio features.
    
#### TRACK IDS PREPARATION

In [2]:
import pandas as pd 
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
# Hid my client id and passport for security and password
cid = 'hidden' 
secret = 'hidden'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
# sp.trace=False 

In [23]:
import timeit
start = timeit.default_timer()

# Empty lists to store results
artist_name = []
track_name = []
track_id = []
uri = []
category = []
popularity = []
# date_release = []

for i in range(0, 2000, 50):  # Current maximum offset is 2000
    results = sp.search(q='year:' + '2020', type='track', limit=50,offset=i)
    for i, t in enumerate(results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        uri.append(t['artists'][0]['uri'])
        category.append(t['type'])
        popularity.append(t['popularity'])
#         date_release.append(t['release_date'])   

stop = timeit.default_timer()
print('Time (in seconds): ', stop - start)

Time (in seconds):  6.730878067999583


In [1]:
# results  

In [24]:
df2020 = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'uri' : uri, 'category' : category, 'popularity' : popularity})
print(df2020.shape)
df2020.head()

# plotly visualization
# collaboratering filtering ( Collaberative Filtering)

(2000, 6)


,artist_name,category,popularity,track_id,track_name,uri
0,SZA,track,92,3YJJjQPAbDT7mGpX3WtQ9A,Good Days,spotify:artist:7tYKF4w9nC0nq9CsPZTHyP
1,Hammock,track,35,51TAp7p7B3xboCl5HWmEkF,Longest Year - 2020,spotify:artist:0VOR7Ie9xUSb45fzIIVJQ1
2,24kGoldn,track,95,3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. iann dior),spotify:artist:6fWVd57NKTalqvmjRd2t8Z
3,Hammock,track,40,79hqdNpnWPVVxmZiDfKaIL,Dark Beyond the Blue - 2020,spotify:artist:0VOR7Ie9xUSb45fzIIVJQ1
4,CJ,track,93,5vGLcdRuSbUhD8ScwsGSdA,Whoopty,spotify:artist:7arQA31aZVS8yS6zUveWzb


I collected a sample of 2000 to begin some analysis of songs and artists with in the year 2020.

Now that I have been able to create a dataframe, I will convert the pandas dataframe to a spark dataframe since I intend to use pyspark for the most part.

In [32]:
sp2020 = spark.createDataFrame(df2020) # creating a spark dataframe from the pandas dataframe I have.
sp2020.show(3)

+-----------+--------+----------+--------------------+--------------------+--------------------+
|artist_name|category|popularity|            track_id|          track_name|                 uri|
+-----------+--------+----------+--------------------+--------------------+--------------------+
|        SZA|   track|        92|3YJJjQPAbDT7mGpX3...|           Good Days|spotify:artist:7t...|
|    Hammock|   track|        35|51TAp7p7B3xboCl5H...| Longest Year - 2020|spotify:artist:0V...|
|   24kGoldn|   track|        95|3tjFYV6RSFtuktYl3...|Mood (feat. iann ...|spotify:artist:6f...|
+-----------+--------+----------+--------------------+--------------------+--------------------+
only showing top 3 rows



Time to persist the data to memory for faster operations.

In [33]:
sp2020.cache()

DataFrame[artist_name: string, category: string, popularity: bigint, track_id: string, track_name: string, uri: string]

In [34]:
sp2020.printSchema()

root
 |-- artist_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- popularity: long (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- uri: string (nullable = true)



In [63]:
# Do I have any duplicates songs in the table? How many if I do?

dup = df2020.drop_duplicates(subset=['artist_name','track_name']).count()[0]
print('Number of duplicates songs are', 2000 - dup)

Number of duplicates songs are 53


In [62]:
# Dropping any duplicates songs in the table

df2020.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
print("Number of songs in table is now", df2020.count()[0])

Number of songs in table is now 1947


In [65]:
# Checking to see if I have any missing values

df2020.isna().sum()

artist_name    0
category       0
popularity     0
track_id       0
track_name     0
uri            0
dtype: int64

Cool no missing values

#### AUDIO FEATURES REPARATION OF THE 1947 SONGS FROM TRACK IDS


In [71]:
start = timeit.default_timer()

rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df2020['track_id']),batchsize):
    batch = df2020['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks with no audio features:', None_counter)

stop = timeit.default_timer()
print('Time (in seconds)::' , stop - start)
print('Number of audio features gotten:', len(rows))

Number of tracks with no audio features: 0
Time (in seconds):: 4.892801713998779
Number of audio features gotten: 1947


In [72]:
df_audio2020 = pd.DataFrame.from_dict(rows, orient='columns')
df_audio2020.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.499,https://api.spotify.com/v1/audio-analysis/3YJJ...,0.4360,279204,0.655,3YJJjQPAbDT7mGpX3WtQ9A,0.000008,1,0.6880,-8.370,0,0.0583,121.002,4,https://api.spotify.com/v1/tracks/3YJJjQPAbDT7...,audio_features,spotify:track:3YJJjQPAbDT7mGpX3WtQ9A,0.4120
1,0.700,https://api.spotify.com/v1/audio-analysis/51TA...,0.0692,533707,0.285,51TAp7p7B3xboCl5HWmEkF,0.673000,2,0.1990,-14.854,1,0.0482,78.572,4,https://api.spotify.com/v1/tracks/51TAp7p7B3xb...,audio_features,spotify:track:51TAp7p7B3xboCl5HWmEkF,0.0312
2,0.221,https://api.spotify.com/v1/audio-analysis/3tjF...,0.7000,140526,0.722,3tjFYV6RSFtuktYl3ZtYcq,0.000000,7,0.2720,-3.558,0,0.0369,90.989,4,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,audio_features,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,0.7560
3,0.729,https://api.spotify.com/v1/audio-analysis/79hq...,0.2000,325680,0.282,79hqdNpnWPVVxmZiDfKaIL,0.887000,2,0.0837,-19.803,0,0.0468,74.381,3,https://api.spotify.com/v1/tracks/79hqdNpnWPVV...,audio_features,spotify:track:79hqdNpnWPVVxmZiDfKaIL,0.0372
4,0.190,https://api.spotify.com/v1/audio-analysis/5vGL...,0.7110,123429,0.598,5vGLcdRuSbUhD8ScwsGSdA,0.000000,3,0.1500,-7.621,0,0.1820,140.175,4,https://api.spotify.com/v1/tracks/5vGLcdRuSbUh...,audio_features,spotify:track:5vGLcdRuSbUhD8ScwsGSdA,0.7370


Getting basic statistics of audio features generated

In [77]:
df_audio2020.type.unique()  # Will drop this column as it has only one entry

array(['audio_features'], dtype=object)

In [88]:
df2020.category.unique()    # Will drop this column also as it has only one entry

array(['track'], dtype=object)

Saving the uri types and removing unnecessaray columns

In [89]:
df_uri = df_audio2020[['analysis_url', 'track_href', 'uri']]

df2020 = df2020.drop(labels=['category'], axis=1)
audio2020 = df_audio2020.drop(labels=['type', 'analysis_url', 'track_href', 'uri'], axis=1)
audio2020.rename(columns={'id': 'track_id'}, inplace=True)
audio2020.head()

,acousticness,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.499,0.4360,279204,0.655,3YJJjQPAbDT7mGpX3WtQ9A,0.000008,1,0.6880,-8.370,0,0.0583,121.002,4,0.4120
1,0.700,0.0692,533707,0.285,51TAp7p7B3xboCl5HWmEkF,0.673000,2,0.1990,-14.854,1,0.0482,78.572,4,0.0312
2,0.221,0.7000,140526,0.722,3tjFYV6RSFtuktYl3ZtYcq,0.000000,7,0.2720,-3.558,0,0.0369,90.989,4,0.7560
3,0.729,0.2000,325680,0.282,79hqdNpnWPVVxmZiDfKaIL,0.887000,2,0.0837,-19.803,0,0.0468,74.381,3,0.0372
4,0.190,0.7110,123429,0.598,5vGLcdRuSbUhD8ScwsGSdA,0.000000,3,0.1500,-7.621,0,0.1820,140.175,4,0.7370


In [76]:
'''
Acousticness: A measure from 0.0 to 1.0 of whether the track is acoustic.

Danceability: Danceability describes how suitable a track is for dancing based on a combination of musical 
elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least 
danceable and 1.0 is most danceable.

duration_ms: Integer typically ranging from 200k to 300k

Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. 
Typically, energetic tracks feel fast, loud, and noisy.

Instrumentalness: Predicts whether a track contains no vocals. The closer the instrumentalness value is to 1.0, 
the greater likelihood the track contains no vocal content.

Liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased 
probability that the track was performed live.

Loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track. 
Values typical range between -60 and 0 db.

Mode (0 = Minor, 1 = Major)

Key (Categorical): All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and 
so on…

Speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the 
recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value.

Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the 
speed or pace of a given piece and derives directly from the average beat duration.

Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high 
valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative 
(e.g. sad, depressed, angry).
'''

In [79]:
audio2020.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
acousticness,1947.0,0.187605,0.270073,0.000001,0.002905,0.050700,0.2560,0.996
danceability,1947.0,0.613074,0.166469,0.000000,0.490500,0.619000,0.7370,0.980
duration_ms,1947.0,187709.338469,81180.626231,30250.000000,146899.500000,188507.000000,223274.5000,842311.000
energy,1947.0,0.686922,0.222871,0.006320,0.550000,0.706000,0.8770,1.000
instrumentalness,1947.0,0.236123,0.363129,0.000000,0.000000,0.000098,0.5310,0.995
key,1947.0,5.329224,3.572187,0.000000,2.000000,6.000000,8.0000,11.000
liveness,1947.0,0.231186,0.185140,0.014700,0.104000,0.147000,0.3175,0.990
loudness,1947.0,-7.649714,3.683460,-34.256000,-8.785000,-6.843000,-5.4650,1.112
mode,1947.0,0.579353,0.493790,0.000000,0.000000,1.000000,1.0000,1.000
speechiness,1947.0,0.111294,0.108608,0.000000,0.042400,0.063200,0.1325,0.933



#### COMBINING (MERGING) BOTH TRACK IDS DATAFRAME AND AUDIO FEATURES DATAFRAME INTO ONE DATAFRAME

In [90]:
df = pd.merge(df2020, audio2020, on='track_id', how='inner')
df.head()

,artist_name,popularity,track_id,track_name,uri,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,SZA,92,3YJJjQPAbDT7mGpX3WtQ9A,Good Days,spotify:artist:7tYKF4w9nC0nq9CsPZTHyP,0.499,0.4360,279204,0.655,0.000008,1,0.6880,-8.370,0,0.0583,121.002,4,0.4120
1,Hammock,35,51TAp7p7B3xboCl5HWmEkF,Longest Year - 2020,spotify:artist:0VOR7Ie9xUSb45fzIIVJQ1,0.700,0.0692,533707,0.285,0.673000,2,0.1990,-14.854,1,0.0482,78.572,4,0.0312
2,24kGoldn,95,3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. iann dior),spotify:artist:6fWVd57NKTalqvmjRd2t8Z,0.221,0.7000,140526,0.722,0.000000,7,0.2720,-3.558,0,0.0369,90.989,4,0.7560
3,Hammock,40,79hqdNpnWPVVxmZiDfKaIL,Dark Beyond the Blue - 2020,spotify:artist:0VOR7Ie9xUSb45fzIIVJQ1,0.729,0.2000,325680,0.282,0.887000,2,0.0837,-19.803,0,0.0468,74.381,3,0.0372
4,CJ,93,5vGLcdRuSbUhD8ScwsGSdA,Whoopty,spotify:artist:7arQA31aZVS8yS6zUveWzb,0.190,0.7110,123429,0.598,0.000000,3,0.1500,-7.621,0,0.1820,140.175,4,0.7370


#### SAVE ALL 3 DATASETS INTO HDFS

In [94]:
# First save all 3 in spark format

sp2020 = spark.createDataFrame(df2020)
sp_audio2020 = spark.createDataFrame(audio2020)
sp = spark.createDataFrame(df)

In [97]:
# Save to HDFS

sp2020.write.save('/user/hadoop/capstone', format='parquet', mode='append')
sp_audio2020.write.save('/user/hadoop/capstone', format='parquet', mode='append')
sp.write.save('/user/hadoop/capstone', format='parquet', mode='append')

In [98]:
# # Save to Local Disk 

df2020.to_csv('SpotifyTracks.csv')
df_audio2020.to_csv('SpotifyAudio.csv')
df.to_csv('Spotify2020.csv')

In [100]:
spark.stop()
sc.stop()